## Assignment 5 - GVFs and Successor Representation
### Jianing Sun 260791202

### - Common Question
**Question:** Explain how SR can be expressed in the GVF framework (also see corresponding section in S&B if necessary). Explain this connection using the GVF terminology : cumulant (this is not the notion of “moments” of a prob. distribution), termination condition, etc. 

**Answer:** In General Value Functions (GVF), whatever signal is added up in a value-function-like prediction, we call it the *cumulant* of that prediction. Denote $C_t\in\mathbb{R}$ as the **cumulant** signal, and **terminal function** $\gamma: \mathcal{S}\mapsto[0,1]$. Then by definition, a **general value function**, or GVF, is written
$$v_{\pi,\gamma,C}(s)=\mathbb{E}\big[\sum\limits_{k=t}^{\infty}C_{k+1}\prod\limits_{i=t+1}^{k}\gamma(S_i)\big|S_t=s,A_{t:\infty}\sim\pi\big]$$ 
Different choices of cumulant signal C lead to different characterizations of the Markov Chain. For example, if C specfies the reward received on entering state s, then $v_{\pi,\gamma,C}(s)$ correponds to the expected discounted future return, or value, the standard target of RL.
In the context of successor representation (SR), **successor representation can be expressed as part of the GVF framework in this way:** 
$$v_{\pi,\gamma,C}(s)=\sum\limits_{s'\in\mathcal{S}}\phi_{\pi}(s,s')C(s)$$
where $\phi_\pi$ is the successor representation (SR), which **encodes the expected discounted future visitations of each state s' along trajectories originating in state s.** Wherein C(s) is the cumulant.

---

### - Trace 1
### 1. Starting from the definition of above, derive Bellman equations for phi_pi. Note : phi_\pi(s) has essentially the meaning of a “value function”.

$ \begin{split} 
\phi_\pi(s)=e_s^T(I-\gamma P_\pi)^{-1}=e_s^T\sum\limits_{t=0}^\infty\gamma^tP_\pi 
=\mathbb{E}\big[e_s^T\sum\limits_{t=0}^\infty\gamma^t\big]
\end{split} $

$\begin{split} 
\Rightarrow \phi_\pi(s,s')&=\mathbb{E}\big[\sum\limits_{t=0}^{\infty}\gamma^t\mathbb{1}(S_{t},s')\big|S_o=s\big] \\
&=\mathbb{E}\big[\mathbb{1}(S_0,s')+\sum\limits_{t=1}^\infty\gamma^t\mathbb{1}(S_{t},s')\big|S_0=s\big] \\
&=\mathbb{E}\big[\mathbb{1}(S_0,s')+\sum\limits_{t=0}^\infty\gamma^{t+1}\mathbb{1}(S_{t+1},s')\big|S_0=s\big] \\
&=\mathbb{E}\big[\mathbb{1}(S_0,s')+\gamma\sum\limits_{t=1}^\infty\gamma^{t}\mathbb{1}(S_{t+1},s')\big|S_0=s\big] \\
&=\mathbb{E}\big[\mathbb{1}(S_0,s')+\gamma\phi_\pi(S_1)\big|S_0=s\big] \\
\end{split} $

where $\mathbb{1}(s,s')$ is an indicator where all elements are zero except 1 for element s'. In other reference [paper](https://papers.nips.cc/paper/5340-design-principles-of-the-hippocampal-cognitive-map.pdf), they usually denote successor representation (SR) with two parameters as M(s,s'). We do not need to marginalize over s' for next step derivation of the successor representation's Bellman Equation. We only need to marginalize over $S_1$:

$
\begin{split}
\phi_\pi(s)&=\sum\limits_{s'}T_{s,s’}\big[\mathbb{1}(s,s')+\gamma\phi_\pi(s')\big] \\
&=\sum\limits_{s'}P(s'|s,a)\sum\limits_a\pi(a|s)\big[\mathbb{1}(s,s')+\gamma\phi_\pi(s')\big] \\
&=\sum\limits_a\pi(a|s)\big[\sum\limits_{s'}P(s'\big|s,a)\mathbb{1}(s,s')+\gamma\sum\limits_{s'}P(s’\big|s,a)\phi_\pi(s')\big]
\end{split}
$

### 2. Derive Bellman equations for state and action dependent phi_\pi(s,a). Let’s call this psi_\pi(s,a) to avoid using the same notation. You can now think of psi_\pi(s,a) as your “Q_\pi(s,a)” and where the “rewards” are vector-valued.

**Answer:** 

assume $\pi$ fixed,

$
\begin{split}
Q^\pi(s,a) &= \mathbb{E}\big[R(S_0,A_0)+\gamma R(S_1,A_1),+...\big|S_0=s,A_0=a, Ai\sim\pi(\cdot|S_i)\big]\\
&=\mathbb{E}\big[\phi(S_0,A_0)w+\gamma\phi(S_1,A_1)w+...\big|S_0=s,A_0=a, Ai\sim\pi(\cdot|S_i)\big]\\
&=\psi^\pi(s,a)\cdot w
\end{split}
$
$\Rightarrow \psi^\pi(s,a)=\mathbb{E}_\pi\big[\sum\limits_{i=1}^\infty\gamma^{i-1}\phi(S_i,A_i)\big|S_0=s,A_0=a\big]$

Where w are parameters to learn.

Special case: for Tabular case, $\phi$ means indicator variables

$\Rightarrow \psi^\pi(s,a)=\phi(s,a)+\gamma\sum\limits_{s'}P(s'|s,a)\sum\limits_{a'}\pi(a'|s')\psi^\pi(s',a')\qquad\qquad\qquad$       (1)

Therefore, Eq.(1) is the bellman equation for $\psi^\pi(s,a)$ which is state-action dependent.

### 3. Show that the “Bellman operator” underlying the evaluation equations in 1. is indeed a contraction. 
Linear Case:

$$\phi_\pi(s,s')=\sum\limits_a\pi(a|s)\big[\sum\limits_{s'}P(s'\big|s,a)\mathbb{1}(s,s')+\gamma\sum\limits_{s'}P(s'|s,a)\phi_\pi(s_1)\big]$$

$$ \forall s, \phi_{k+1}(s) \leftarrow \sum\limits_a\pi(a|s)\big(\sum\limits_{s'}P(s'\big|s,a)\mathbb{1}(s,s')+\gamma\sum\limits_{s'}P(s'\big|s,a)\phi_k(s')\big) $$

$\begin{split}
\phi_{k+1}(s)-\phi_\pi(s)&=\sum\limits_a\pi(a|s)\sum\limits_{s'}P(s'\big|s,a)\mathbb{1}(s,s')+\sum\limits_a\pi(s|a)\gamma\sum\limits_{s'}P(s'\big|s,a)\phi_k(s')\\
&-\sum\limits_a\pi(a|s)\sum\limits_{s'}P(s'\big|s,a)\mathbb{1}(s,s')+\sum\limits_a\pi(s|a)\gamma\sum\limits_{s'}P(s'\big|s,a)\phi_\pi(s')\\
&=\gamma\sum\limits_a\pi(a|s)\sum\limits_{s'}P(s'\big|s,a)\big[\phi_k(s')-\phi_\pi(s')\big] \\
&\leq\gamma\sum\limits_a\pi(a|s)\sum\limits_{s'}P(s'\big|s,a)\max\limits_{s'}\big[\phi_k(s')-\phi_\pi(s')\big]
\end{split}$

denote $\varepsilon_{k+1}(s)=\phi_{k+1}(s)-\phi_\pi(s)$
$$\varepsilon_k(s)=\big|\big|\phi_k(s)-\phi_\pi(s)\big|\big|_{k\rightarrow\infty}$$

Hence, 

$\begin{split}
\phi_{k+1}(s)-\phi_\pi(s)&\leq\gamma\sum\limits_a\pi(a|s)\sum\limits_{s'}P(s'\big|s,a)\max\limits_{s'}\big[\phi_l(s')-\phi_\pi(s')\big] \\
&\leq\gamma\varepsilon_k\sum\limits_a\pi(a|s)\sum\limits_{s'}P(s'\big|s,a)\\
&\leq\gamma\varepsilon_k
\end{split}$

That is $\varepsilon_{k+1}\leq\gamma\epsilon_k$, and $\gamma \in (0,1)$

$$\Rightarrow \varepsilon_{k+1}\leq\gamma^k\varepsilon_0 \stackrel{k\rightarrow\infty}{\longrightarrow}0$$

Therefore, it is indeed a contraction Bellman operator. Non-linear case is the same derivation process as the one in linear case, just under the maximization condition.